In [ ]:
import wandb
api = wandb.Api()

# Just load seaborn & set theme and the chart looks better:
! pip install seaborn -q
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import numpy as np
import matplotlib.colors as cor
from collections import defaultdict 
import pandas as pd
sns.set_theme()

In [ ]:
_runs_attn = api.runs("seperability/new-method-compare")
_runs_orig = api.runs("seperability/pile-code-attn")
_runs = [*_runs_attn]#, *_runs_orig]
wandb_cache = {}
_reverse = False

In [ ]:
metric_map = {"base": "Top1", "topk": "Top10", "skip": "Skip50-Top1", "topk_skip": "Skip50-Top10", "loss": "Loss", "perplexity": "Perplexity"}
color_map  = {"pile": "tab:orange", "code": "tab:blue", "python": "tab:green"}
dataset_map = {"pile": "Pile", "pile_codeless": "Pile", "python": "Python", "code": "Code"}
fmt_map = {
    "opt-125m": ":yo",
    "opt-1.3b": "--yo",
    "opt-6.7b": "-yo",
    "galactica-125m": ":C2s",
    "galactica-1.3b": "--C2s",
    "galactica-6.7b": "-C2s",
    "pythia-160m": ":C4x",
    "pythia-1.4b": "--C4x",
    "pythia-6.9b": "-C4x",
    "roberta-large": ":C5x",
}

cripple_or_focus = "Retain" if _reverse else "Forget"
def test_skip_rev(name):
    if (_reverse and not "rev" in name) and not "random" in name:
        return True
    if (not _reverse and "rev" in name):
        return True
    return False


def df_append(df, item: dict):
    new_data = pd.DataFrame({ k:[v] for k,v in item.items() })
    df = pd.concat([ df, new_data ], ignore_index=True )
    return df

def is_loss_metric(metric_name):
    if metric_name == "perplexity":
        return True
    if metric_name[-4:] == "loss":
        return True
    return False

def normed(h, dataset, key):
    if key == "perplexity":
        return np.exp(h[f"loss_data/{dataset}/loss"])/np.exp(h[f"loss_data/{dataset}/loss"][0])
    if is_loss_metric(key):
        return h[f"loss_data/{dataset}/{key}"]/h[f"loss_data/{dataset}/{key}"][0]
    return h[f"accuracy/{dataset}/{key}"]/h[f"accuracy/{dataset}/{key}"][0]

def calculate_area(x, y):
    return np.trapz(y, x)

def calculate_max_diff(x, y, _reversed=False):
    x, y = np.array(x), np.array(y)
    diff = (x-y) if _reversed else (y-x)
    return diff.max()

class WandbRunData:
    def __init__(self, run_name):
        self.run_name = run_name.split("https://wandb.ai/")[-1]
        
        if self.run_name in wandb_cache:
            self.run = api.run(self.run_name)
        else:
            self.run = api.run(self.run_name)
            wandb_cache[run_name] = self.run
        
        self.history = self.run.history()
        self.h = self.history
        c = self.run.config
        
        self.model_name = c["model_size"].split("/")[-1].lower()
        
        max_frac = max( c["ff_frac"], c["attn_frac"] )
        self.frac = self.history["_step"] * max_frac
        
        self.cripple, self.focus = c["cripple"], c["focus"]
        self.cripple_label = dataset_map[self.cripple]
        self.focus_label   = dataset_map[self.focus]
        
        # Get "unique" names
        self.name_set = self.focus_label + " " + self.cripple_label
        self.name_set_model = self.name_set + " " + self.model_name
        
    def get_metric(self, metric, diff):
        if is_loss_metric(metric):
            return self.get_loss_metric(metric)
        if diff:
            return self.get_diag_metric(metric)
        return self.get_d_metric(metric)
   
    def get_loss_metric(self, metric):
        self.scale = 1
        focus_perf     = normed(self.h, self.focus,   metric) * self.scale
        cripple_perf   = normed(self.h, self.cripple, metric) * self.scale

        d_focus, d_cripple = focus_perf, cripple_perf
        #area = calculate_area(focus_perf, cripple_perf)*2 / (self.scale**2)
        use_reverse = _reverse and "random" in self.run.name
        print(use_reverse, self.run.name)
        score = calculate_max_diff(focus_perf, cripple_perf, _reversed=use_reverse)
        return focus_perf, cripple_perf, score
    
    def get_diag_metric(self, metric):
        self.scale = 100
        focus_perf     = normed(self.h, self.focus,   metric) * self.scale
        cripple_perf   = normed(self.h, self.cripple, metric) * self.scale
        
        d_cripple, d_focus = self.scale-cripple_perf, self.scale-focus_perf
        #area = ( calculate_area(d_focus, d_cripple) - self.scale**2/2 )/(self.scale**2)
        use_reverse = _reverse and "random" in self.run.name
        print(use_reverse, self.run.name)
        score = calculate_max_diff(d_focus, d_cripple, _reversed=use_reverse)
        return d_cripple, d_focus, score
     
    def get_d_metric(self, metric):
        self.scale = 100
        focus_perf     = normed(self.h, self.focus,   metric) * self.scale
        cripple_perf   = normed(self.h, self.cripple, metric) * self.scale
        
        d_cripple, d_focus = self.scale-cripple_perf, self.scale-focus_perf
        #area = calculate_area(d_focus, d_cripple-d_focus)*2 / (self.scale**2)
        use_reverse = _reverse and "random" in self.run.name
        print(use_reverse, self.run.name)
        score = calculate_max_diff(d_focus, d_cripple, _reversed=use_reverse)
        return d_cripple, d_focus, score


In [ ]:
def plot_horizontal(_data, scale=1.0):
    # Data sort
    print(scale)
    _data.sort()
    num_points = len(_data)
    _min, _max  = min([_data[ 0][0],  0.0]), max([_data[-1][0], 1.0*scale])
    #if _min < 0.0:
    #    _min = -1*scale
    linspace = np.linspace(_min, _max, num_points+2)
    
    # Create a figure and axis
    fig, (ax2, ax1) = plt.subplots(ncols=2, figsize=(2.5, 6),
        gridspec_kw={'width_ratios': [6, 15]}, dpi=300)
    plt.subplots_adjust(wspace=0)
    
    # Remove the y-axis and set the y-limits
    ax1.get_yaxis().set_visible(False)
    ax1.get_xaxis().set_visible(False)
    ax1.set_frame_on(False)
    ax1.set_xlim(0, 1)
    ax1.hlines(0,     0, 1, color="k", linestyle="--", alpha=0.001)
    ax1.hlines(scale, 0, 1, color="k", linestyle="--", alpha=0.001)
        
    
    ax2.set_xlim(0, 1)
    ax2.get_xaxis().set_visible(False)
    ax2.hlines(0,     0, 1, color="k", linestyle="-", linewidth=0.4)
    ax2.hlines(scale, 0, 1, color="k", linestyle="-", alpha=0.8)
    
    # Plot the points on the number line
    for index, (point, label) in enumerate(_data):
        # Default
        linestyle = "-."
        color = None 
        fontweight = "normal"
        fontstyle = "normal"
        
        if "ff" in label:
            linestyle = "-"
        elif "attn h" in label:
            linestyle = ":"
        elif "attn" in label:
            linestyle = "--"
           
        if "random" in label:
            #fontweight = "bold"
            color = "k"
            
        #if "rev" in label:
        #    fontstyle = "italic"
       
        y_label = linspace[index+1] 
        ax1.plot([0, 0.15, 0.25], [point, y_label, y_label], linestyle=linestyle, color=color)
        ax1.text(0.3, y_label-0.005*scale, label, rotation=0, ha='left', va='bottom',
                 fontstyle=fontstyle, fontsize=12, fontweight=fontweight)
        
        ax2.plot([0,1], [point, point], linestyle=linestyle, color=color)
 
    return fig, (ax1, ax2)

def plot_technique_comparison(runs_filtered, scale=1.0, **kwargs):   
    df = pd.DataFrame()
    seen = defaultdict(list)
    for run in runs_filtered:
        r = WandbRunData(run)
        x = 1-normed(r.h, r.cripple, "base")
        y = 1-normed(r.h, r.focus,   "base")
        x = np.array([*x, 1])
        y = np.array([*y, 1])
        #area =  1 - calculate_area(x, y)*2
        use_reverse = _reverse and "random" in r.run.name
        score = np.array(calculate_max_diff(y, x, _reversed=use_reverse))
        
        if not "attn_prune_heads" in r.run.config:
            r.run.config["attn_prune_heads"] = False
        if not "attn_prune_heads_mode" in r.run.config:
            r.run.config["attn_prune_heads_mode"] = r.run.config["attn_prune_heads"]
            
        # Get "unique" names
        name = r.run.name.lower()
        name = name.split("gal 1.3b")[-1]
        name = name.split("pythia 1.4b")[-1]
        name = name.split("pc opt 1.3b")[-1]
        name = name.split("opt 1.3b")[-1]
        name = "h".join(name.split("head"))
        #name = "".join(name.split(" orig"))
        name = "".join(name.split(" v"))
        name = "".join(name.split(" orig"))
        name = "".join(name.split(" rev"))
        
        if "attn h" in name:
            h_type = "attn head " + r.run.config["attn_prune_heads_mode"]
            name = h_type.join( name.split("attn h") )
        #if "rev" in name:
        #    score *= -1
        if r.run.config["attn_prune_heads_mode"] == "median":
            continue
       
        c = r.run.config 
        gen_name = ""
        if not "ff_scoring" in c:
            c["ff_scoring"] = "freq"
        if not "attn_scoring" in c:
            continue
        if c["ff_frac"] > 0 and c["attn_frac"] > 0:
            gen_name += "both"
        elif c["ff_frac"] > 0:
            gen_name += "ff"   + " " + c["ff_scoring"]
        elif c["attn_frac"] > 0:
            gen_name += "attn"
            if c["attn_prune_heads"]:
                gen_name += " head " + c["attn_prune_heads_mode"]     
            gen_name += " " + c["attn_scoring"]
        
        
        #if name in already_seen:
        #    continue
        if name.lower().strip() != gen_name.lower().strip():
            print(f"possibly mislabelled '{gen_name}' (was written as '{name}'). Skipping")
        #    continue
            
        seen[gen_name].append( (score, r.run.name, c["ff_frac"], c["attn_frac"]) ) 
       
    #    plt.plot([0, 1], [1, area], label = r.run.name)
    for name, datum in seen.items():
        (val, orig_name, ff_frac, attn_frac) = list(sorted(datum))[-1]
        new_data = {
            "name": name,
            "score": val,
            "orig_name": orig_name,
            "ff_frac": ff_frac,
            "attn_frac": attn_frac,
        }
        df = df_append(df, new_data)
    if len(df) == 0:
        return None
    df = df.sort_values(by="score", ascending=False)
    display(df)
    l = len(df)
   
    data = [ (row["score"]*scale, row["name"]) for _i, row in df.iterrows() ]

    return plot_horizontal(data, scale, **kwargs)
    
scale = 100
points = np.random.random(15) * 100
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']
labels = ["Point "+x for x in labels]
data = list(zip(points, labels))
data.sort()
print(data)

fig, (ax1, ax2) = plot_horizontal(data, scale)
ax1.set_ylim(-0*scale, 1.01*scale)
ax2.set_ylim(-0*scale, 1.01*scale)
#ax2.text(-1, -0.5, "Code Focus",   ha="left",  va="top")
#ax2.text( 1, -0.5, "Code Cripple", ha="right", va="top")
plt.show()

In [ ]:
runs_filtered, run_limit = [], None
for run in _runs:
    if run_limit and len(runs_filtered) > run_limit:
        break
    if run.state == "crashed":
        continue
    if not "model_size" in run.config:
        continue
    if not run.config["model_size"] == "facebook/galactica-1.3b":
        continue
    if not "_step" in run.summary or run.summary["_step"] < 1:
        continue
    #if not "attn" in run.name:
    #    continue
    if run.config["attn_frac"] > 0 and run.config["attn_frac"] < 0.02:
        continue
    if test_skip_rev(run.name):
        continue
    if "svd" in run.name and "attn h" in run.name:
        continue
    if "svd" in run.name:
        continue
    if "sqrt" in run.name:
        continue
    if "both" in run.name:
        continue
    if "few" in run.name:
        continue
    if "L" in run.name:
        continue
    if run.name in [
            "gal 1.3b del bias attn abs",
            "gal 1.3b attn",
            "gal 1.3b both old",
            "gal 1.3b both svd-b",
            "gal 1.3b ff",
        ]:
        continue
    #print(run.name, run.config.items())
    runs_filtered.append(run.url)

scale = 100
plot_out = plot_technique_comparison(runs_filtered, scale)
if plot_out is not None:
    fig, (ax1, ax2) = plot_out
    ax1.set_ylim(-0.01*scale, 1.01*scale)
    ax2.set_ylim(-0.01*scale, 1.01*scale)
    #ax2.text( 1, -0.55, "Code Cripple", ha="right", va="top")
    ax2.set_ylabel(f"Galactica 1.3B - Code {cripple_or_focus} Top1 Max ∆ (∆%)")

    plt.show()

In [ ]:
runs_filtered, run_limit = [], None
for run in _runs:
    if not "model_size" in run.config:
        continue
    if not run.config["model_size"].lower() == "EleutherAI/pythia-1.4B".lower():
        continue
    if run_limit and len(runs_filtered) > run_limit:
        break
    if not "_step" in run.summary or run.summary["_step"] < 1:
        continue
    #if not "attn" in run.name:
    #    continue
    if _reverse and "attn h" in run.name:
        continue
    if test_skip_rev(run.name):
        continue
    if "base" in run.name:
        continue
    if "sqrt" in run.name:
        continue
    if run.config["ff_frac"] > 0 and run.config["ff_frac"] < 0.02:
        continue
    #if "attn" in run.name:
    #    continue
    if "L" in run.name:
        continue
    if run.name in [
            "gal 1.3b del bias attn abs",
            "gal 1.3b attn",
            "gal 1.3b both old",
            "gal 1.3b both svd-b",
        ]:
        continue
    #print(run.name, run.config.items())
    runs_filtered.append(run.url)
   
scale = 100 
plot_out = plot_technique_comparison(runs_filtered, scale)
if plot_out is not None:
    fig, (ax1, ax2) = plot_out
    ax1.set_ylim(-0.01*scale, 1.01*scale)
    ax2.set_ylim(-0.01*scale, 1.01*scale)
    ax2.set_ylabel(f"Pythia 1.4B - Code {cripple_or_focus} Top1 Max ∆ (∆%)")
    plt.show()

In [ ]:

runs_filtered, run_limit = [], None
for run in _runs:
    if run_limit and len(runs_filtered) > run_limit:
        break
    if run.state == "crashed":
        continue
    if not "_step" in run.summary or run.summary["_step"] < 1:
        continue
    #if not "pc" in run.name:
    #    continue
    print(run.name)
    if not "model_size" in run.config:
        print(run.name)
        continue 
    if test_skip_rev(run.name):
        continue
    if " r " in run.name:
        continue
    if "svd" in run.name:
        continue
    if "single" in run.name:
        continue
    if "sqrt" in run.name:
        continue
    if _reverse and "attn h" in run.name:
        continue
    #if "svd" in run.name and "attn h" in run.name:
    #    continue
    #if "attn" in run.name and False and \
    #        not ("attn v" in run.name or "attn h" in run.name):
    #    continue
    if not run.config["model_size"] == "facebook/opt-1.3b":
        continue
    if " L" in run.name:
        continue
    if run.name in [
            "gal 1.3b del bias attn abs",
            "gal 1.3b attn",
            "gal 1.3b both old",
            "gal 1.3b both svd-b",
            "gal 1.3b ff",
        ]:
        continue
    #print(run.name, run.config.items())
    runs_filtered.append(run.url)

scale = 100
plot_out = plot_technique_comparison(runs_filtered, scale)
if plot_out is not None:
    fig, (ax1, ax2) = plot_out
    ax1.set_ylim(-0.01*scale, 1.01*scale)
    ax2.set_ylim(-0.01*scale, 1.01*scale)
    ax2.set_ylabel(f"OPT-1.3b - Code {cripple_or_focus} Top1 Max ∆ (∆%)")

    plt.show()

In [ ]:

runs_filtered, run_limit = [], None
for run in _runs:
    if run_limit and len(runs_filtered) > run_limit:
        break
    if run.state == "crashed":
        continue
    if not "_step" in run.summary or run.summary["_step"] < 1:
        continue
    #if not "pc" in run.name:
    #    continue
    print(run.name)
    if not "model_size" in run.config:
        print(run.name)
        continue 
    if test_skip_rev(run.name):
        continue
    if " r " in run.name:
        continue
    if "svd" in run.name:
        continue
    if "single" in run.name:
        continue
    if "sqrt" in run.name:
        continue
    if _reverse and "attn h" in run.name:
        continue
    #if "svd" in run.name and "attn h" in run.name:
    #    continue
    #if "attn" in run.name and False and \
    #        not ("attn v" in run.name or "attn h" in run.name):
    #    continue
    if not run.config["model_size"] == "roberta-large":
        continue
    #print(run.name, run.config.items())
    runs_filtered.append(run.url)

scale = 100
plot_out = plot_technique_comparison(runs_filtered, scale)
if plot_out is not None:
    fig, (ax1, ax2) = plot_out
    ax1.set_ylim(-0.01*scale, 1.01*scale)
    ax2.set_ylim(-0.01*scale, 1.01*scale)
    ax2.set_ylabel(f"RoBERTa - Code {cripple_or_focus} Top1 Max ∆ (∆%)")

    plt.show()